<a href="https://colab.research.google.com/github/elemi10/7506-TP-Org-de-datos/blob/7506-Trabajo-Practico-1/TP1_Mas_ordenado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':'1RAGDjlzJ6spO5Sq8_x3UTIvxLhKAUBEt'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv') 

### Importo **Librerias**

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
import datetime as dt
import scipy as sp
#from sklearn.cross_validation import train_test_split -- no levanta
%matplotlib inline
plt.rcParams['figure.figsize'] = (16,9)
plt.style.use('default') # mejoramos esteticamente un poco los gráficos en matplotlib
sns.set(style='whitegrid') # seteando tipo de grid en seaborn

from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
#from sklearn.metrics import confusion_matrix
#from sklearn.naive_bayes import GaussianNB
#from sklearn.metrics import classification_report
#from sklearn.feature_selection import SelectKBest
#from sklearn.cross_validation import train_test_split --> No levanta el feature de la libreria

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Levantramos el archivo **train.csv**

In [16]:
df_train = pd.read_csv(r'train.csv', usecols=['keyword','location','text','target']) 
df_train.head()

,keyword,location,text,target
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [17]:
# Verifico que no haya instancias nulas o filas completas nulas
df_train=df_train.dropna(how="all")
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7613 entries, 0 to 7612
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keyword   7552 non-null   object
 1   location  5080 non-null   object
 2   text      7613 non-null   object
 3   target    7613 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 297.4+ KB


In [18]:
# Normalizo el dataframe los registros NaN con texto
df_train=df_train.fillna({'keyword': 'sin keyword',\
                   'location': 'sin location'})
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7613 entries, 0 to 7612
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keyword   7613 non-null   object
 1   location  7613 non-null   object
 2   text      7613 non-null   object
 3   target    7613 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 297.4+ KB


In [19]:
# Eliminamos los tweets duplicados considerando que su cantidad no tiene impacto en el análisis.
df_train=df_train.drop_duplicates('text')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 0 to 7612
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keyword   7503 non-null   object
 1   location  7503 non-null   object
 2   text      7503 non-null   object
 3   target    7503 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 293.1+ KB


In [20]:
# Verificamos la existencia de valores nulos
df_train.isnull().any()

keyword     False
location    False
text        False
target      False
dtype: bool

In [21]:
# Calculamos unas estadisticas basicas
df_train.describe()

,target
count,7503.000000
mean,0.426230
std,0.494561
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [22]:
# cuento total de tweets analizados
total_tweets=len(df_train)
total_tweets

7503

In [23]:
# verifico cuantos son verdaderos
tweets_verdaderos=(df_train['target']==1).sum()
tweets_verdaderos

3198

In [24]:
# verifico cuantos son fakes
tweets_falsos=(df_train['target']==0).sum()
tweets_falsos

4305

In [25]:
# calculo la probabilidad (casos favorables/sobre casos posibles) → en este casos es equivalente .mean()
probabilidad_tweet_verdadero = np.round((tweets_verdaderos)/(total_tweets),decimals=5)
probabilidad_tweet_verdadero

0.42623

In [26]:
# calculo la probabilidad (casos favorables/sobre casos posibles) → en este casos es equivalente .mean()
probabilidad_tweet_falso = np.round((tweets_falsos)/(total_tweets),decimals=2)
probabilidad_tweet_falso

0.57

### Preprocesamiento de datos

#### Creamos la variable para **contar la cant de caracteres del tweet**

In [0]:
# Creo una columna con el cálculo de las palabras que tiene la columna 'text' y la llamo 'caracteres_tweet'. 
# Despues consulto con un head()
df_train['caracteres_tweet']=df_train['text'].str.len()
df_train.head(20)